In [13]:
import requests
import os
import json
from minio import Minio
import io

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.environ.get("TWITTER_BEARER_TOKEN")

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r


def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()


def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


def set_rules(delete):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "dog has:images", "tag": "dog pictures"},
        {"value": "cat has:images -grumpy", "tag": "cat pictures"},
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))


def get_stream(set):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    i = 0
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            formatted = json.dumps(json_response, indent=4, sort_keys=True)
            print(formatted)
            s3_client.put_object(bucket, f"test_tweets/{i}.json", io.BytesIO(bytes(formatted, "utf-8")), len(formatted))



In [14]:
s3_access_key = os.environ.get("S3_ACCESS_KEY")
s3_secret_key = os.environ.get("S3_SECRET_KEY")
bucket = "twitter-data"
s3_client = Minio("localhost:9000", access_key=s3_access_key, secret_key=s3_secret_key, secure=False)

if not s3_client.bucket_exists(bucket):
    print(f"creating bucket '${bucket}'")
    s3_client.make_bucket(bucket)
else:
    print(f"bucket already exists")

SyntaxError: expected ':' (4179357296.py, line 9)

In [3]:
rules = get_rules()
delete = delete_all_rules(rules)
set = set_rules(delete)


{"data": [{"id": "1486142647643217923", "value": "dog has:images", "tag": "dog pictures"}, {"id": "1486142647643217924", "value": "cat has:images -grumpy", "tag": "cat pictures"}], "meta": {"sent": "2022-01-26T01:59:40.439Z", "result_count": 2}}
{"meta": {"sent": "2022-01-26T01:59:40.743Z", "summary": {"deleted": 2, "not_deleted": 0}}}
{"data": [{"value": "cat has:images -grumpy", "tag": "cat pictures", "id": "1486156857785786368"}, {"value": "dog has:images", "tag": "dog pictures", "id": "1486156857785786369"}], "meta": {"sent": "2022-01-26T01:59:41.065Z", "summary": {"created": 2, "not_created": 0, "valid": 2, "invalid": 0}}}


In [12]:
get_stream(set)

200
{
    "data": {
        "id": "1486169754364645383",
        "text": "I have a cat baby  at my house \ud83c\udfe0  Look at my #BinanceCat he always try hide herself from my family members but she so familiar with me&amp; my niece . At  dinner time she always ate with me  unfortunately I lost her 14 January. Now I'm feel alone at dinner miss u \"Lukhi\"\ud83d\ude41\n#Binance https://t.co/twoQ59udP9 https://t.co/k9jJ9y6c9P"
    },
    "matching_rules": [
        {
            "id": "1486156857785786369",
            "tag": "dog pictures"
        },
        {
            "id": "1486156857785786368",
            "tag": "cat pictures"
        }
    ]
}
{
    "data": {
        "id": "1486169756977987585",
        "text": "RT @dog_haru73: \u3053\u308c\u3059\u304d https://t.co/ZUVrRLa5wd"
    },
    "matching_rules": [
        {
            "id": "1486156857785786369",
            "tag": "dog pictures"
        }
    ]
}
{
    "data": {
        "id": "1486169758395416578",
        "text": "

{
    "data": {
        "id": "1486169803580600321",
        "text": "Lila cat is very upset with me. She heard that computers often come with mice and wants to know why I have a portable trackpad instead. #CatsofTwittter #phdstudent #bruinpets https://t.co/CzyQQ4lGC0"
    },
    "matching_rules": [
        {
            "id": "1486156857785786368",
            "tag": "cat pictures"
        }
    ]
}
{
    "data": {
        "id": "1486169792935436290",
        "text": "RT @dog_poongjin: \uc6c3\ub294 \uac1c\ub294 \uc815\ub9d0 \uadc0\uc5ec\uc6cc https://t.co/ZfCSUyNoPF"
    },
    "matching_rules": [
        {
            "id": "1486156857785786369",
            "tag": "dog pictures"
        }
    ]
}
{
    "data": {
        "id": "1486169811428130820",
        "text": "RT @jisoupy: when the cat gets into bed before you and u gotta sleep in a weird position https://t.co/KPBvUEm1ve"
    },
    "matching_rules": [
        {
            "id": "1486156857785786368",
            "tag": "cat p

KeyboardInterrupt: 